# D2E2F: Helsingborg-Helsingør
(2021-04-23)

In [ ]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)
plt.style.use('presentation')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize, animate
from src.data import get_dataset
from src.data import prepare_dataset
from src.data import trips


import scipy.integrate

In [ ]:
df = get_dataset.get(name='tycho_short_id')

## Plot trips

In [ ]:
visualize.plot_map(df=df, width=1400, height=800, zoom_start=14, color_key='trip_direction')

## Trip statistics

In [ ]:
df_stat = get_dataset.trip_statistics()

In [ ]:
df_stat_plot = df_stat.copy()

df_stat_plot['energy per trip [Nm]'] = df_stat_plot['power_em_thruster_total']

df_stat_plot['trip_direction'] = df_stat_plot['trip_direction'].apply(lambda x : 'Helsingør-Helsingborg' if x==0 else 'Helsingborg-Helsingør')

fig = px.scatter(df_stat_plot, x='start_time',y='energy per trip [Nm]', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'])
fig.show()

## Analyzing trips

In [ ]:
fig = px.line(df, x='trip_time', y='sog', color='trip_no', width=1500, height=800)
fig.show()

## Look at one trip

In [ ]:
trips = df.groupby(by='trip_no')
trip_no = 110
#trip_no = 3

trip = trips.get_group(trip_no)

trip_ = trip.copy()
trip_ = prepare_dataset.calculate_rudder_angles(df=trip_, drop=False)
trip_['trip_time_s'] = pd.TimedeltaIndex(trip_['trip_time']).total_seconds()

power_columns = ['power_em_thruster_%i' % i for i in range(1,5)]
trip_[power_columns]/=trip_['power_em_thruster_total'].max()/4

trip_ = trip_.resample('2S').mean()


In [ ]:
visualize.plot_map(df=trip_, width=1400, height=800, zoom_start=14, color_key='trip_direction')

In [ ]:
animate.widget(trip=trip)

In [ ]:
import imageio

animation_1_dir = 'anmation1'
if not os.path.exists(animation_1_dir):
    os.mkdir(animation_1_dir)

file_paths = []
trip_anim = trip_.resample('5S').mean()
for index, row in trip_anim.iterrows():
    plot_thrust_allocation(row=row)
    file_name = '%i.png' % row['trip_time_s']
    file_path = os.path.join(animation_1_dir, file_name)
    file_paths.append(file_path)
    plt.savefig(file_path)
    plt.close()
    


In [ ]:
save_file_path = os.path.join(animation_1_dir,'trip_%i.gif' % trip_no)
with imageio.get_writer('mygif.gif', mode='I') as writer:
    for file_path in file_paths:
        image = imageio.imread(file_path)
        writer.append_data(image)
        
for file_path in file_paths:
    os.remove(file_path)

## Is there a more optimal thruster operation?

![](https://memegenerator.net/img/images/15921176.jpg)

### 1) Utveckla simuleringsmodell med Machine Learning
### 2) Använda simuleringsmodell för att optimera thrustrar 


### Analysarbete
#### a) Långtidsanalys: dagar/veckor/månader/år
#### b) Förbättring med "Smart Eco-shipping"
#### c) Korttidsanalys: farygsdynamik under resa
